<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/fine-tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/train.txt'
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/test.txt'
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/dev.txt'
!mkdir data && mv train.txt data && mv test.txt data && mv dev.txt data

train.txt           100%[===================>]   2.20M  --.-KB/s    in 0.04s   
test.txt            100%[===================>] 108.10K  --.-KB/s    in 0.003s  
dev.txt             100%[===================>] 160.83K  --.-KB/s    in 0.003s  


# Benchmark2

In [3]:
!cat data/train.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > train.txt.tmp
!cat data/test.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > test.txt.tmp
!cat data/dev.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > dev.txt.tmp

In [4]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 62958 (delta 26), reused 60 (delta 14), pack-reused 62875
Receiving objects: 100% (62958/62958), 48.07 MiB | 28.94 MiB/s, done.
Resolving deltas: 100% (44579/44579), done.


In [5]:
!pip -q install transformers/
!pip -q install sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2MB 22.3MB/s 
     |████████████████████████████████| 890kB 51.8MB/s 
     |████████████████████████████████| 1.2MB 16.0MB/s 


In [6]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

--2021-02-20 03:53:57--  https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 991 [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      

2021-02-20 03:53:57 (59.3 MB/s) - ‘preprocess.py’ saved [991/991]



In [7]:
!python3 preprocess.py dev.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > dev.txt
!python3 preprocess.py train.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > train.txt
!python3 preprocess.py test.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > test.txt

Downloading: 100% 434/434 [00:00<00:00, 638kB/s]
Downloading: 100% 1.22M/1.22M [00:00<00:00, 36.2MB/s]


In [8]:
!pip -q install -r transformers/examples/token-classification/requirements.txt

     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 184kB 27.9MB/s 
     |████████████████████████████████| 102kB 14.6MB/s 
     |████████████████████████████████| 245kB 56.0MB/s 
     |████████████████████████████████| 20.7MB 1.5MB/s 


In [9]:
!pip -q install conllu

In [10]:
!mkdir processed_data
!mv train.txt processed_data/ && mv test.txt processed_data/ && mv dev.txt processed_data/

In [11]:
!cat processed_data/train.txt processed_data/test.txt processed_data/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

In [15]:
!python3 transformers/examples/legacy/token-classification/run_tf_ner.py --help

2021-02-20 03:57:02.220360: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
usage: run_tf_ner.py [-h] --model_name_or_path MODEL_NAME_OR_PATH
                     [--config_name CONFIG_NAME] [--task_type TASK_TYPE]
                     [--tokenizer_name TOKENIZER_NAME] [--use_fast [USE_FAST]]
                     [--cache_dir CACHE_DIR] --data_dir DATA_DIR --labels
                     LABELS [--max_seq_length MAX_SEQ_LENGTH]
                     [--overwrite_cache [OVERWRITE_CACHE]]
                     [--output_dir OUTPUT_DIR]
                     [--overwrite_output_dir [OVERWRITE_OUTPUT_DIR]]
                     [--do_train [DO_TRAIN]] [--do_eval [DO_EVAL]]
                     [--do_predict [DO_PREDICT]]
                     [--evaluation_strategy {no,steps,epoch}]
                     [--prediction_loss_only [PREDICTION_LOSS_ONLY]]
                     [--per_device_train_batch_size PER_DEVICE_TRAIN_BATCH_SIZ

In [27]:
!python3 transformers/examples/legacy/token-classification/run_tf_ner.py \
--data_dir ./processed_data/ \
--labels labels.txt \
--model_name_or_path HooshvareLab/bert-base-parsbert-uncased \
--output_dir eval/ \
--max_seq_length  256 \
--per_device_train_batch_size 8 \
--save_steps 1000 \
--save_total_limit 1 \
--eval_steps 100 \
--logging_strategy steps \
--logging_steps 100 \
--logging_first_step True \
--evaluation_strategy steps \
--seed 1 \
--num_train_epochs 5 \
--do_train \
--do_eval \
--do_predict

2021-02-20 04:41:43.287043: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
[INFO|training_args.py:578] 2021-02-20 04:41:44,547 >> PyTorch: setting up devices
[INFO|training_args.py:516] 2021-02-20 04:41:44,585 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[INFO|training_args_tf.py:185] 2021-02-20 04:41:44,590 >> Tensorflow: setting up strategy
2021-02-20 04:41:44.590823: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-20 04:41:44.590915: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-02-20 04:41:44.591053: I tensorflow/stream_executor/cuda/cuda_gpu_executor.c

In [26]:
!rm -rf runs eval

In [30]:
!zip -r eval.zip eval

  adding: eval/ (stored 0%)
  adding: eval/checkpoint/ (stored 0%)
  adding: eval/checkpoint/ckpt-4.index (deflated 82%)
  adding: eval/checkpoint/checkpoint (deflated 40%)
  adding: eval/checkpoint/ckpt-4.data-00000-of-00001 (deflated 30%)
  adding: eval/tf_model.h5 (deflated 8%)
  adding: eval/test_predictions.txt (deflated 71%)
  adding: eval/tokenizer_config.json (deflated 45%)
  adding: eval/eval_results.txt (deflated 27%)
  adding: eval/test_results.txt (deflated 64%)
  adding: eval/special_tokens_map.json (deflated 40%)
  adding: eval/config.json (deflated 54%)
  adding: eval/vocab.txt (deflated 62%)


In [29]:
!zip -r runs.zip runs

  adding: runs/ (stored 0%)
  adding: runs/Feb20_04-41-44_279572b6a5c4/ (stored 0%)
  adding: runs/Feb20_04-41-44_279572b6a5c4/events.out.tfevents.1613796123.279572b6a5c4.905.4401.v2 (deflated 72%)


In [31]:
!ls -lh

total 1.9G
drwxr-xr-x  2 root root 4.0K Feb 20 03:53 data
-rw-r--r--  1 root root 161K Feb 20 03:53 dev.txt.tmp
drwxr-xr-x  3 root root 4.0K Feb 20 05:22 eval
-rw-r--r--  1 root root 1.9G Feb 20 05:26 eval.zip
-rw-r--r--  1 root root   74 Feb 20 03:54 labels.txt
-rw-r--r--  1 root root  991 Feb 20 03:53 preprocess.py
drwxr-xr-x  2 root root 4.0K Feb 20 03:54 processed_data
drwxr-xr-x  3 root root 4.0K Feb 20 04:42 runs
-rw-r--r--  1 root root 7.1K Feb 20 05:22 runs.zip
drwxr-xr-x  1 root root 4.0K Feb 16 16:35 sample_data
-rw-r--r--  1 root root 109K Feb 20 03:53 test.txt.tmp
-rw-r--r--  1 root root 2.3M Feb 20 03:53 train.txt.tmp
drwxr-xr-x 15 root root 4.0K Feb 20 03:53 transformers


In [32]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
!mv eval.zip drive/MyDrive/

In [23]:
!mv runs.zip drive/MyDrive/